In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
df = pd.read_csv('/Users/sanjayroberts1/Desktop/data_science_tools1/sudeste.csv')

In [9]:
df = df.set_index('mdct')
df.index = pd.to_datetime(df.index)

In [16]:
d = {}
for col in df:
    d[col] = df[col].unique().tolist()

In [30]:
print(d['gbrd'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
df.shape

(9779168, 30)

In [20]:
print(df.isna().sum())

wsid          0
wsnm          0
elvt          0
lat           0
lon           0
inme          0
city          0
prov          0
date          0
yr            0
mo            0
da            0
hr            0
prcp    8371184
stp           0
smax          0
smin          0
gbrd    4108820
temp         31
dewp        475
tmax         26
dmax        310
tmin         34
dmin        807
hmdy          0
hmax         12
hmin         44
wdsp     925561
wdct          0
gust     316474
dtype: int64


Some stations do not have wind or rain sensors, owing to why there are so many Null values. Will set them to zero to be able to do cleanup, then separate stations to do individual analysis

Gbrd has null values for when the sun is down (little less than half the time which makes sense). Make these zero values also to not affect analysis.

Other column null values we can set to zero and interpolate to get a value.

- Separate data for each station
- Each station may have it's own issues for data cleanup, so we separate and apply a cleanup function for each

In [22]:
df['prcp'].fillna(0,inplace=True)

In [31]:
df['gbrd'].fillna(0,inplace=True)

In [33]:
df.fillna(0,inplace=True)

In [34]:
df.isna().sum()

wsid    0
wsnm    0
elvt    0
lat     0
lon     0
inme    0
city    0
prov    0
date    0
yr      0
mo      0
da      0
hr      0
prcp    0
stp     0
smax    0
smin    0
gbrd    0
temp    0
dewp    0
tmax    0
dmax    0
tmin    0
dmin    0
hmdy    0
hmax    0
hmin    0
wdsp    0
wdct    0
gust    0
dtype: int64

Drop where all sensor columns are 0

In [36]:
col = ['prcp', 'stp', 'smax', 'smin', 'gbrd', 'temp',
       'dewp', 'tmax', 'dmax', 'tmin', 'dmin', 'hmdy', 'hmax', 'hmin', 'wdsp',
       'wdct', 'gust']
df = df[(df[col] != 0).any(axis=1)]


In [37]:
df.shape

(9159199, 30)

9779168 - 9159199 = 619969 columns dropped

take care of all but wind data. Look closer into wind data issues

In [39]:
df['temp'].replace(0, np.nan, inplace=True)
df['temp'].interpolate('time', inplace=True, limit_direction='both')
df['dewp'].replace(0, np.nan, inplace=True)
df['dewp'].interpolate('time', inplace=True, limit_direction='both')
df['tmax'].replace(0, np.nan, inplace=True)
df['tmax'].interpolate('time', inplace=True, limit_direction='both')
df['tmin'].replace(0, np.nan, inplace=True)
df['tmin'].interpolate('time', inplace=True, limit_direction='both')
df['dmax'].replace(0, np.nan, inplace=True)
df['dmax'].interpolate('time', inplace=True, limit_direction='both')
df['dmin'].replace(0, np.nan, inplace=True)
df['dmin'].interpolate('time', inplace=True, limit_direction='both')
df['hmax'].replace(0, np.nan, inplace=True)
df['hmax'].interpolate('time', inplace=True, limit_direction='both')
df['hmin'].replace(0, np.nan, inplace=True)
df['hmin'].interpolate('time', inplace=True, limit_direction='both')

Count number of nonzeros in each column

In [40]:
df.astype(bool).sum(axis=0)

wsid    9159199
wsnm    9159199
elvt    9119489
lat     9119489
lon     9119489
inme    9159199
city    9159199
prov    9159199
date    9159199
yr      9159199
mo      9159199
da      9159199
hr      8778203
prcp     661305
stp     9077445
smax    9073730
smin    9073810
gbrd    4790858
temp    9159199
dewp    9159198
tmax    9159199
dmax    9159199
tmin    9159199
dmin    9159199
hmdy    9099495
hmax    9159199
hmin    9159199
wdsp    8025423
wdct    8908786
gust    8621514
dtype: int64

Count number of zeros in each column

In [44]:
(df == 0).astype(int).sum(axis=0)

wsid          0
wsnm          0
elvt      39710
lat       39710
lon       39710
inme          0
city          0
prov          0
date          0
yr            0
mo            0
da            0
hr       380996
prcp    8497894
stp       81754
smax      85469
smin      85389
gbrd    4368341
temp          0
dewp          1
tmax          0
dmax          0
tmin          0
dmin          0
hmdy          0
hmax          0
hmin          0
wdsp    1133776
wdct     250413
gust     537685
dtype: int64

In [43]:
df['hmdy'].replace(0, np.nan, inplace=True)
df['hmdy'].interpolate('time', inplace=True, limit_direction='both')

look into why elevation, lat, lon is zero (all same weather station?), same with smin, smax etc, and look at surrornding values to see if it makes sense

hr, prcp, and gbrd are ok to be 0

https://stackoverflow.com/questions/40660088/get-first-row-of-dataframe-in-python-pandas-based-on-criteria

then work on wind stuff to clean that data up

In [45]:
df.to_hdf('cleaning_up_df.hdf','mydata',mode='w')